# Implicit solutions to coupled systems of ODEs

With the linear algebra techniques of solving coupled systems of equations we can now construct solutions to coupled ODEs. As an example we will write a nuclear network code. Such networks exhibit an ODE behaviour that is called _stiff_. Coefficients in stiff ODE systems have a very large range, which implies that the explicit solution scheme we have used so far would demand taking very small time steps to calculate details of the solutions that are not relevant. The solution is to adopt an implicit scheme. It involves numerical root-finding with the Newton-Raphson method. 

* Newton-Raphson root finding
* implicit solution of ODEs using Newton-Raphson
* solving coupled systems of ODEs

## Root finding: Newton-Raphson method

**Literature: ** Numerical Recipes, Chapter 9.0 and 9.4 

Root-finding is required in many applications of computational physics. It is at the core of the fundamental task to solve an equation numerically. Any equation with a right-hand and left-hand side can be rearranged to 
$$ f(x) = 0.$$
If the there is only one independent variable we have just a one-dimensional problem. The Newton-Raphson method is a robust and easy method to find the root of a function. It has very fast convergence if there is a good initial guess, but it has - like other methods - several pathologies that one needs to be aware off. One of its advantages is that it does generalize to multiple dimensions in a straight-forward way. 

### Mathematical idea of the method

The method starts with a good initial guess $x_{ini}$ of the root. Evaluating the function and the derivative for the initial guess provides a correction $\delta$ toward the actual root $x'$.

![nr-method](../images/NR_scheme.jpg)

We can then repeat this improvement until a termination criterion certifies that the solution is good enough. 

### Algorithm
Once we have the basic idea and mathematical formulation of the method in place, and before we actually start the coding it often is useful to write our strategy down in the form of an algorithm.

![nr-algorithm](../images/NR_algorithm.jpg)


### Termination condition

The termination conditions determine if a solution is good enough to
be accepted. Really, we are checking on the precision with which the
solution is satisfying the discretized equation. The termination
conditions for the NR method may include the following:

- $\delta < \delta_{lim}$: corrections are smaller than some limit
- the function evaluation is smaller than some limit, i.e. close to zero: $f(x_n)< \epsilon_{fmax}$ 

### Pathological cases

This termination condition can check on a number of things, and the
particular choices may depend on the particular problem. Before
writing down the termination conditions it makes sense to anticipate
pathological cases, i.e. situations in which this method will fail:

- higher-order terms are important, and initial guess is not good enough $\rightarrow$ the method will send you off to infinity
- non-convergent cycles: there could be a case, for example, when table look-up is involved in calculating the derivative, that correction are bouncing the guess from 
one side of the root to the other indefinitely without ever coming closer
- there could be multiple roots, and the one the NR method is finding is only a local root selected by the choice of initial guess.

Note that all methods have pathologies, or in other words, have their
particular weaknesses and strength. Selecting the most appropriate
method for a particular problem requires you to consider the
advantages and disadvantages of different methods.


The following remedies can be added to the termination conditions to accomodate the pathological cases:

- corrections are monotonically decreasing
- limit maximum number of iterations 
- restart iterations with different initial guess


### Example
Let's explore some of the pathologies that can be encountered with an example.

$$ f(x) = x^3 + 3x^2 -3 $$

In [ ]:
%pylab nbagg 

In [ ]:
# We will need the function 
f = lambda x: x**3 +3*x**2-3

# and the derivative
def derivs(f,x,h):
    '''
    Return 1st order numerical derivative
    
    f : function
    x : x value to evaluate derivative
    h : interval in x for derivative
    '''
    dfdx = (f(x+h) - f(x)) /h
    return dfdx
   
 

In [ ]:
# visual inspection
close(1); figure(1)
x = linspace(-3,1.5,400)
plot(x, f(x))
axhline(color='k',linestyle='dashed')

In [ ]:
h = 0.01
# different cases for xx
xx = 0.6
#xx = -0.2
#xx = -2.01
plot(xx,f(xx),'*')

In [ ]:
# do on NR iteration
x = xx
delta = -f(x)/derivs(f,x,h)
xx = x + delta
plot(xx,f(xx),'h')
# repeat?

In [ ]:
xx, f(xx)


## Implicit ODE with Newton-Raphson
The Newton-Raphson method can be used as well for an alternative
approach to integrating ODEs, such as the one that appeared in the sky diver
problem. So far we have used an _explicit_ forward integration which
involves evaluating the RHS of the ODE using the know velocity
$ v_n $ at the present time to get the value of the
velocity at the next time step $ n+1 $. In cases where
we are interested in the longer-term behaviour, and where we are
required to take larger time steps this may not work very well. In
that case we want to evaluate the RHS for $ v(n+1) $, and we have
an _implicit_ solution scheme. Especially for large coupled systems of
ODEs implicit schemes often provide the numerical stability to find a
solution at all. The implicit solution involves an iterative approach,
and we can use again the Newton-Raphson method:
![nr-method](../images/NR_for_ODEs.jpg)

As you can see the basic strategy is to put everything on one side,
give it a name $ G(v_{n+1})$, set this function to zero and
solve it by Newton-Raphson. The beauty is that this idea can be easily
expanded to systems of ODEs with many variables. In physics this 
method is for example used for large reaction rate networks involving
thousands of isotopes, solving radiation transfer problems or computing the evolution of stars in spherical symmetry according to the conservation laws of mass, momentum and energy.

Let's implement the skydiver problem as an implicit solution. Recall that the differential equation was just $$ \frac{dv}{dt} = -g +  k v^2$$ and we may add the equation for the height of the skydiver $$ \frac{dh}{dt} = v$$ (see Lecture 2 in Week 4). The explicit solution scheme is $$ v_{n+1} = v_{n} +dt( -g +  k v_{n}^2),$$ whereas the implicit solution would be $$ v_{n+1} = v_{n} +dt( -g +  k v_{n+1}^2).$$

#### Explicit:

In [ ]:
k = 0.0022; g = 9.8
v_t = sqrt(g/k)
h   = 2000   # jump-off height [m]
h_p = 300    # end of free flight, pull parachute height [m]

In [ ]:
  # typical spatial step at terminal velocity [m]
  # time step?

In [ ]:
def rhs_sdiver(x,dt):
    'Evaluate RHS for skydiver problem, advance one time step'
 
    return [h,v]

In [ ]:
hv_all = []
hv = array([h,0])

# integrate until termination condition

figure(3)
plot(hv_10[0],hv_10[1])
xlim(2100,250)
xlabel('h / [m]')
ylabel('v / [m/s]')

In [ ]:
# v(t) instead of v(h)
# maximum time? 
t_max = dt*(h-h_p)/dh

In [ ]:
dh = [10,100, 300]
close(2)
figure(2)
for this_dh in dh:
    dt = this_dh / v_t
    tv_all = [] # each element is [t, v]
    hv = array([h,0]); t = 0
    while t < t_max:
        hv = rhs_sdiver(hv,dt)
        t += dt
        tv_all.append([t,hv[1]])
    tv = array(tv_all).T
    plot(tv[0],tv[1],label='dh='+str(this_dh)+'m')
xlabel('t / [s]')
ylabel('v / [m/s]')
legend(loc=1)


## Implicit solution
Next we consider the **implicit** solution, which involves finding the root of $$ v_{n+1} = v_{n} +dt( -g +  k v_{n+1}^2)$$ where in each time step $v_{n+1}$ is the independent variable. In other words, if $$ G(v_{n+1}) =  - v_{n+1} + v_{n} +dt( -g +  k v_{n+1}^2)$$ then find for each time step $ G(v_{n+1}) = 0$.


In [ ]:
G = ... 


In [ ]:
# plot result
close(4)
figure(4)
x=linspace(-5,5)
plot(x,G(x))
axhline(color='k',linestyle='dashed')

In [ ]:
xx=0; h=0.01

In [ ]:
# one NR iteration step
x = xx
...

In [ ]:
# do all time steps, each as a number of NR iterations
while tt < t_max:


In [ ]:
figure(2) 
plot(t,v,':',label='implicit_lr')
legend()

## Implicit solution for systems of ODEs

If we do want to know the height as well as the position of the sky diver then we have a coupled system of ODEs, in discretized, inmplicit form:

$$ v_{n+1} = v_{n} +dt( -g +  k v_{n+1}^2)\\
h_{n+1} = h_{n} +dt v_{n+1}$$

and in terms of the function we have to find the root for

$$ G_1(v_{n+1}) = - v_{n+1} + v_{n} +dt( -g +  k v_{n+1}^2)\\
G_2(h_{n+1}) = -h_{n+1} + h_{n} +dt v_{n+1}$$
 
and with $Y_1 = v_{n+1}$ and $Y_2 = h_{n+1}$ and $c_1 = v_{n} - dt\, g$ and $c_2 = h_{n}$

$$ G_1(Y_1) = - Y_1 + c_1 + dt\,k\, Y_1^2\\
G_2(Y_2) = -Y_2 + c_2 + dt\, Y_1 $$

or 

$$ \vec{G}(\vec{Y}) = \matrix{F} \cdot \vec{Y} + \vec{c}\\
\matrix{F} = \pmatrix{ (-1 + dt\,k) & 0  \\ 
                       dt & -1  }
$$ 

Now the Newton-Raphson for finding the root of $\vec{G}(\vec{Y})=0$ would look like this

$$
\vec{G}'(\vec{Y}) \, \vec{\delta} = -\vec{G}(\vec{Y}) 
$$

where \vec{Y} is to start with the estimate of the solution for the next time step, and $\vec{G}'(\vec{Y})$ is the Jacoby matrix. It has on our case the following elements:

$$
\vec{G}'(\vec{Y}) = \pmatrix{ \frac{\partial G_1}{Y_1} & \frac{\partial G_1}{Y_2}  \\ 
                     \frac{\partial G_2}{Y_1} & \frac{\partial G_2}{Y_2}  }
                     = \matrix{F}
$$
and so, for each time step we have to do a number of Newton-Raphson iterations, each of which involves solving the linear algebra problem

$$
\matrix{F} \cdot \vec{\delta} = -\vec{G}(\vec{Y}) 
$$

which we can solve, i.e. find $\vec{\delta}$, using Gaussian elimination.

More generally, for a system with $N$ ODEs we would have $N$ variables:

$$\vec{Y} =\left(\begin{array}{c}
Y_1 \\
Y_2 \\
\vdots \\
Y_i \\
\vdots \\
Y_N \\
\end{array}\right)
$$

The Jacoby matrix is 
$$
\matrix{F}=\frac{\partial \vec{f}(\vec{Y}^{n})}{\partial \vec{Y}^{n}}
=\left(\begin{array}{c}
\vec{f}_1'(\vec{Y}) \\
\vec{f}_2'(\vec{Y}) \\
\vdots \\
\vec{f}_i'(\vec{Y}) \\
\vdots \\
\vec{f}_N'(\vec{Y})
\end{array}\right) ,
$$
and contains the row elements
$$
  \frac{\partial f_\mathrm{i}(\vec{Y}) }  {\partial \vec{Y}}=
  \vec{f}_i'(\vec{Y})=\left( \frac{\partial f_i}{\partial Y_{1}},
    \frac{\partial f_i}{\partial Y_{2}},\dots,
      \frac{\partial f_i}{\partial Y_{i}},\dots,
      \frac{\partial f_i}{\partial Y_{N}}\right) .
$$
which defines for each iteration in each time step a linear algebra problem with $N$ variables.

In many real problems the Jacoby matrix may be _sparse_ which means it is only partially populated and lots of elements are zero. In that case special numerical solution techniques are used, e.g. _sparse solvers_